In [ ]:
import os
import sys
import numpy as np
import keras as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from dataloader import load_cifar

### Set env

In [ ]:
# TensorFlow,Theano,CNTK
os.environ['KERAS_BACKEND'] = "tensorflow" #Use TF1,some incompatibilities with TF2.
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Performance Improvement
# Make sure channels-first (not last)
K.backend.set_image_data_format('channels_first')

### Load dataset

In [ ]:
x_train, x_test, y_train, y_test = load_cifar(channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

### Init model

In [ ]:
# Hyperparams
EPOCHS  =  10
BATCHSIZE  =  64
LR  =  0.01
MOMENTUM  =  0.9
N_CLASSES  =  10
GPU  =  True
BATCH_SIZE  =  32

def create_model(n_classes=N_CLASSES):
    model = Sequential()
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu',
                     input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

def init_model(m, lr=LR, momentum=MOMENTUM):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(lr, momentum),
        metrics = ['accuracy'])
    return m

model = create_model()
model = init_model(model)
model.summary()

### Main training loop

In [ ]:
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

### Main evaluation loop

In [ ]:
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))